In [1]:
from monai.transforms import LoadImaged, EnsureChannelFirstd, Compose
from src.transforms import (
    FindCentroid,
    GetFixedROISize,
    CropToCentroidWithSize,
    PadForCrop,
    Save,
    OrthogonalSlices,
    CropToROI,
    GetZoomedROI,
    PadForZoom,
)
from pathlib import Path

def find_corresponding_scan(segmention_path):
    scan_folder = Path(segmention_path).parent.parent / "scans"
    scan_name = Path(segmention_path).name.split(".")[0][:-2] + ".nii.gz"

    return scan_folder / scan_name

pipeline = Compose([
    LoadImaged(keys=["img", "seg"]),
    EnsureChannelFirstd(keys=["img", "seg"]),
])


segmentation_path = r'D:\premium_data\lumc\monotherapy\split_segmentations\PREM_LU_009_0.nii.gz'
scan_path = find_corresponding_scan(segmentation_path)
data = {'img':scan_path, 'seg':segmentation_path}

output = pipeline(data)


In [5]:
select = Compose(
    [
        FindCentroid(),
        GetFixedROISize(10),
        PadForCrop(),
        CropToCentroidWithSize(),
    ]
)

roi = select(output)

In [6]:
roi['centroid']

array([222, 269, 522])

In [1]:
import wandb
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Trainer, seed_everything
from src.model import Model
from src.data import DataModule

wandb.init()

wandb.config.aggregation_function = 'max'
wandb.config.method = 'crop'
wandb.config.dim = 3
wandb.config.size = 128
wandb.config.roi_size= 10
wandb.config.test_center = 'amphia'
wandb.config.lesion_target = 'lung'
wandb.config.max_batch_size = 8
wandb.config.seed = 0
wandb.config.max_epochs = 10

dm = DataModule(
    r"C:\Users\user\data\dl_radiomics",
    r"C:\Users\user\data\tables\lesion_followup_curated_v4.csv",
    wandb.config
)

model = Model(wandb.config)

seed_everything(wandb.config.seed)

logger = WandbLogger(
    name="hello_3",
    project="project_skeleton_on_lung_lesions",
)

trainer = Trainer(
    max_epochs=wandb.config.max_epochs, gpus=1, deterministic=True, fast_dev_run=False, logger=logger
)

trainer.fit(model, dm)

2022-10-31 11:29:11,681 - Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: renstermaat. Use `wandb login --relogin` to force relogin


Columns (28,33,186,190,236,248,252,371,376,433,470,480,525,532,535,543,544,546,549,551,552,553,573,574,575,576,581,585,655,706,752,761) have mixed types.Specify dtype option on import or set low_memory=False.
Global seed set to 0


2022-10-31 11:29:14,633 - Global seed set to 0


GPU available: True, used: True


...

2022-10-31 11:29:14,760 - LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]




  | Name              | Type        | Params
--------------------------------------------------
0 | model             | DenseNet121 | 11.2 M
1 | train_auc         | BinaryAUROC | 0     
2 | val_auc           | BinaryAUROC | 0     
3 | train_patient_auc | BinaryAUROC | 0     
4 | val_patient_auc   | BinaryAUROC | 0     
--------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.975    Total estimated model params size (MB)


2022-10-31 11:29:16,382 - 
  | Name              | Type        | Params
--------------------------------------------------
0 | model             | DenseNet121 | 11.2 M
1 | train_auc         | BinaryAUROC | 0     
2 | val_auc           | BinaryAUROC | 0     
3 | train_patient_auc | BinaryAUROC | 0     
4 | val_patient_auc   | BinaryAUROC | 0     
--------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.975    Total estimated model params size (MB)
Validation sanity check:  50%|█████     | 1/2 [00:02<00:02,  2.14s/it]

No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score


Epoch 0:  71%|███████▏  | 40/56 [00:38<00:15,  1.04it/s, loss=0.648, v_num=yws9, valid_loss=0.688, valid_auc=0.525, valid_patient_auc=0.450]